Connecting to the Database and clean the data 

Data structure explanation: 

junctionID - a unique ID number assigned to whole junction

approachID – a unique ID number assigned to this specific approach line

travelTimeSec - information on how long (in seconds) it currently takes for the vehicles to travel through the approach

freeFlowTravelTime - information on how long (in seconds) it takes for the vehicles to travel through the approach during free-flow conditions( when there is no traffic jams, etc)

delaySec - the current delay (in seconds) 

usualDelaySec - this is the usual delay that is expected at this time of day

Stops - the average number of stops per vehicle on the approach

queueLengthMeters - information on how long (in meters) the current line of vehicles waiting on the junction approach is

volumePerHour - an estimated value of how many vehicles we observe traveling through the approach per hour

isClosed – information if the approach was closed for traffic 

stopsHistogram - a container that stores histogram data specific to stops ( example: 0,3 is displaying the data like this: "numberOfStops": 0, "numberOfVehicles": 3) 




Fistly, I connected to the Database through SQLAlchemy engine and get the 'approaches' and 'traffic-main' tables. Then, I cleaned 'approaches' table. After cleaning I merged 2 tables data (especially, all the columns from 'approaches' table and only 'direction' column from 'traffic-main' table) using JOIN function. Finally, I store all the data in new table called 'updated_approaches'.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
db_params = {
    'dbname': 'traffic-system',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost',
    'port': '5432'
}

# Create a connection string
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@" \
                    f"{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Step 1: Load data from 'approaches' table into a DataFrame
query_approaches = "SELECT * FROM approaches;"
df = pd.read_sql(query_approaches, engine)

# Step 2: Data Cleaning
# Drop rows with any missing values
df.dropna(inplace=True)

# Drop rows where 'stopshistogram' is null
df.dropna(subset=['stopshistogram'], inplace=True)

# Drop the 'junctionid' column if it exists
df.drop(columns=['junctionid'], inplace=True, errors='ignore')

# Replace 'approaches' table with cleaned data
df.to_sql('approaches', engine, if_exists='replace', index=False)

# Step 3: Join cleaned data with 'traffic_main' on 'approachid' and 'id'
query_join = """
SELECT a.*, t.direction
FROM approaches a
JOIN traffic_main t ON a.approachid = t.id;
"""
df_joined = pd.read_sql(query_join, engine)

# Step 4: Save the joined DataFrame to a new table 'updated_approaches'
df_joined.to_sql('updated_approaches', engine, if_exists='replace', index=False)

# Display the first few rows and the number of rows in the joined DataFrame
print(df_joined.head())
print(f"Number of rows in the joined DataFrame: {df_joined.shape[0]}")

engine.dispose()


Now, connect to the database again and merge the data from 'weather.csv' and the data from 'updated_approaches' table in 'traffic-system' DB. 

To do that I created a new columns in  'updated_approaches' that will store the time data (especially, the time data excluding minutes and seconds). Then, as above, I JOINed 2 tables and stored that table I got from the JOIN to the new table called 'merged_traffic_weather'. 

In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection parameters
db_params = {
    'dbname': 'traffic-system',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost',
    'port': '5432'
}

# Create a connection string
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@" \
                    f"{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Step 1: Load data from 'updated_approaches' table
query_updated_approaches = "SELECT * FROM updated_approaches;"
df_updated_approaches = pd.read_sql(query_updated_approaches, engine)

# Step 2: Create a new column 'rounded_time' that floors 'time' to the nearest hour
df_updated_approaches['rounded_time'] = pd.to_datetime(df_updated_approaches['time']).dt.floor('H')

# Step 3: Load weather data from CSV and convert 'time' to datetime format in UTC
weather_file_path = '/Users/asylkaziahmetova/Desktop/bit/traffic-management/data/weather.csv'
df_weather = pd.read_csv(weather_file_path)
df_weather['time'] = pd.to_datetime(df_weather['time'], format='%Y-%m-%dT%H:%M').dt.tz_localize('UTC')

# Step 4: Merge the DataFrames on 'rounded_time' and 'time' from weather data
df_merged = pd.merge(df_updated_approaches, df_weather, left_on='rounded_time', right_on='time', how='inner')

# Step 5: Save the merged DataFrame to a new table in the database
df_merged.to_sql('merged_traffic_weather', engine, if_exists='replace', index=False)

# Display the first few rows and the number of rows in the merged DataFrame
print(df_merged.head())
print(f"Number of rows in the merged DataFrame: {df_merged.shape[0]}")

# Close the engine connection
engine.dispose()


ModuleNotFoundError: No module named 'psycopg2'

Correlation analysis.

To find the pairwise relationships between the selected features a correlation matrix was created. Then heatmap visualization of this matrix was used for accessing. Overall, I accessed the correlation of the Traffic Volume and Weather data correlations.  


In [13]:
# Select only the relevant columns for correlation analysis
df_corr = df_merged[['delaysec', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 
                     'apparent_temperature (°C)', 'rain (mm)', 'wind_speed_10m (km/h)', 
                     'stops','time_x','volumeperhour', 'queuelengthmeters']]
correlation_matrix = df_corr.corr()


NameError: name 'df_merged' is not defined

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
correlation_matrix = df_corr.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", 
            vmin=-1, vmax=1, fmt=".2f", linewidths=0.5)

plt.title("Correlation Matrix of Traffic Delay and Weather Features")
plt.show()


NameError: name 'df_corr' is not defined

Split the data into train and test (80 and 20%) 

In [ ]:
from sklearn.model_selection import train_test_split

X = df_merged[['temperature_2m (°C)', 'relative_humidity_2m (%)', 
                'apparent_temperature (°C)', 'rain (mm)', 
                'wind_speed_10m (km/h)', 'stops', 
                'queuelengthmeters']]
y = df_merged['volumeperhour']  

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.2, random_state=42)
print(X_train.shape) 
print(y_train.shape) 


Train the RandomForestRegressor model and fit the train data 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)


Use the model to predict on the test data and make evaluations. 

In [ ]:
from sklearn.metrics import r2_score

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f" R2: {r2}")

Smart traffic light timing adjustment using predictions made by the RandomForestRegressor model. 

In [ ]:
def adjust_traffic_lights(predictions, initial_timings={'Green': 60, 'Yellow': 5, 'Red': 30}):
    traffic_light_timings = []

    for volume in predictions:
        green_time = initial_timings['Green']
        yellow_time = initial_timings['Yellow']
        red_time = initial_timings['Red']

        if volume < 100:
            green_time = 60
        elif 100 <= volume < 500:
            green_time = 50
        elif 500 <= volume < 1000:
            green_time = 40
        elif 1000 <= volume < 3000:
            green_time = 30
        else:  # volume >= 3000
            green_time = 20

        traffic_light_timings.append(f"Green: {green_time} seconds, Yellow: {yellow_time} seconds, Red: {red_time} seconds")

    return traffic_light_timings

predictions = y_pred[:100] 
traffic_light_adjustments = adjust_traffic_lights(predictions)

for i, adjustment in enumerate(traffic_light_adjustments):
    print(f"Prediction {i + 1}: Volume = {predictions[i]}, Adjustments = {adjustment}")


Additional model to update the traffic light based on each line (on a crossroad) 

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import time

# Database connection parameters
db_params = {
    'dbname': 'traffic-system',
    'user': 'postgres',
    'password': '1234',
    'host': 'localhost',
    'port': '5432'
}

# SQLAlchemy connection string
connection_string = f"postgresql://{db_params['user']}:{db_params['password']}@" \
                    f"{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
engine = create_engine(connection_string)

# Initial timing (in seconds) for each direction
initial_green_time = {
    'NORTH': 30,
    'SOUTH': 30,
    'EAST': 30,
    'WEST': 30
}

# Threshold parameters
THRESHOLD_TRAVEL_RATIO = 1.2  # 20% over free-flow time
THRESHOLD_DELAY_RATIO = 1.2  # 20% over usual delay
GREEN_TIME_INCREMENT = 10  # seconds

# Step 1: Initialize Traffic Light Timing
traffic_light_timings = initial_green_time.copy()

# Function to detect heavy traffic
def is_heavy_traffic(row):
    traffic_condition = (
        row['traveltimesec'] > row['freeflowtraveltimesec'] * THRESHOLD_TRAVEL_RATIO or
        row['delaysec'] > row['usualdelaysec'] * THRESHOLD_DELAY_RATIO
    )
    return traffic_condition

# Step 2 & 3: Detect and Adjust Traffic Light Timing Based on Congestion
def adjust_traffic_lights():
    # Fetch the data from the database
    query = "SELECT * FROM updated_approaches;"
    df = pd.read_sql(query, engine)
    
    # Group data by timestamp
    grouped_by_timestamp = df.groupby('time')
    
    for timestamp, group in grouped_by_timestamp:
        congested_directions = {}

        # Check each row in the timestamp group for heavy traffic
        for _, row in group.iterrows():
            # Avoid division by zero by setting a minimum value of 1 for these columns
            freeflow_time = row['freeflowtraveltimesec'] if row['freeflowtraveltimesec'] != 0 else 1
            usual_delay = row['usualdelaysec'] if row['usualdelaysec'] != 0 else 1
            
            if is_heavy_traffic(row):
                direction = row['direction']
                congestion_level = (
                    (row['traveltimesec'] / freeflow_time) +
                    (row['delaysec'] / usual_delay)
                )
                congested_directions[direction] = congestion_level

        # Step 4: Adjust the green light for the most congested direction
        if congested_directions:
            most_congested_direction = max(congested_directions, key=congested_directions.get)
            traffic_light_timings[most_congested_direction] += GREEN_TIME_INCREMENT
            print(f"Adjusted green light for {most_congested_direction}: {traffic_light_timings[most_congested_direction]} seconds")

    return traffic_light_timings

# Simulation loop for real-time adjustments
def run_traffic_light_system():
    while True:
        current_timings = adjust_traffic_lights()
        print("Current traffic light timings:", current_timings)
        
        # Sleep for the cycle duration before the next adjustment
        time.sleep(10)

# Start the traffic light system
run_traffic_light_system()
